In [1]:
# packages gerais
import pandas as pd
import joblib

# df_sales_ dataset
df_sales_filtered = joblib.load('df_sales_filtered.pkl')

In [2]:
df_sales_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1382485 entries, 757 to 8815748
Data columns (total 28 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   ID                     1382485 non-null  int64         
 1   store_id               1382485 non-null  object        
 2   product_id             1382485 non-null  object        
 3   date                   1382485 non-null  datetime64[ns]
 4   sales                  1382485 non-null  float64       
 5   revenue                1382485 non-null  float64       
 6   stock                  1382485 non-null  float64       
 7   price                  1382485 non-null  float64       
 8   promo_type_1           1382485 non-null  object        
 9   promo_bin_1            1382485 non-null  object        
 10  promo_type_2           1382485 non-null  object        
 11  promo_bin_2            1382485 non-null  object        
 12  promo_discount_2       1382485 